# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import yaml

import feisty

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
testcase = feisty.driver.simulate_testcase("tanh_shelf", "cyclic")

In [4]:
with open("matlab_first_timestep.yaml") as f:
    baselines = yaml.safe_load(f)


def compare(baselines_dict, da_or_ds, full_table=True, markdown_formatting=False):
    if markdown_formatting:
        if "dimname" in baselines_dict:
            print(f"| {baselines_dict['dimname']} | Matlab Value | Python Value | Rel Err |")
        else:
            print(f"|  | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ["| ", " | ", " | ", " |"]
    else:
        seps = ["", ": ", ", ", ""]
    if "dimname" in baselines_dict:
        all_vars = da_or_ds[baselines_dict["dimname"]].data
    else:
        all_vars = baselines_dict["values"]
    for var in all_vars:
        if "dimname" in baselines_dict:
            da = da_or_ds
            sel_dict = {baselines_dict["dimname"]: var}
        else:
            da = da_or_ds[var]
            sel_dict = {}
        matlab_val = baselines_dict["values"][var]
        py_val = da.sel(sel_dict).data
        if matlab_val != 0:
            rel_err = (py_val - matlab_val) / matlab_val
            if rel_err < 0:
                rel_err = -rel_err
        else:
            if py_val == 0:
                rel_err = 0
            elif py_val > 0:
                rel_err = np.inf
            elif py_val < 0:
                rel_err = -np.inf
        if full_table or rel_err > 1e-2:
            print(
                f"{seps[0]}{var}{seps[1]}{matlab_val:10.4e}{seps[2]}"
                + f"{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}"
            )

In [5]:
compare(
    baselines["day1"]["forcing"],
    testcase.forcing.isel(time=0, X=0, zooplankton=0),
    markdown_formatting=True,
)

|  | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| T_pelagic | 1.8200e+01 | 1.8200e+01 | 0.0000e+00 |
| T_bottom | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| zooC | 3.3287e+00 | 3.3287e+00 | 0.0000e+00 |
| poc_flux_bottom | 2.7847e-02 | 2.7847e-02 | 1.2459e-16 |
| zoo_mort | 7.7564e-01 | 7.7564e-01 | 0.0000e+00 |


## Run the model


In [6]:
%%time

nsteps = 2
make_plot = False
testcase.run(nsteps)
testcase.ds

CPU times: user 483 ms, sys: 3.13 ms, total: 486 ms
Wall time: 511 ms


<xarray.Dataset>
Dimensions:                    (X: 22, group: 9, time: 2, fish: 8, feeding_link: 19)
Coordinates:
  * X                          (X) float64 -0.5 -0.2381 0.02381 ... 4.738 5.0
  * group                      (group) <U12 'Sf' 'Sp' ... 'Ld' 'benthic_prey'
  * fish                       (fish) <U2 'Sf' 'Sp' 'Sd' 'Mf' ... 'Md' 'Lp' 'Ld'
  * feeding_link               (feeding_link) <U15 'Sf_Zoo' ... 'Ld_benthic_p...
    predator                   (feeding_link) <U2 'Sf' 'Sp' 'Sd' ... 'Ld' 'Ld'
    prey                       (feeding_link) <U12 'Zoo' ... 'benthic_prey'
Dimensions without coordinates: time
Data variables: (12/16)
    biomass                    (time, group, X) float64 9.999e-06 ... 0.0003619
    T_habitat                  (time, fish, X) float64 18.2 18.2 ... 18.2 18.2
    ingestion_rate             (time, fish, X) float64 0.2152 ... 7.131e-07
    predation_flux             (time, fish, X) float64 5.737e-12 ... 0.0
    predation_rate             (time, fish, X) float64 5.737e-07 ... 0.0
    metabolism_rate            (time, fish, X) float64 0.04296 ... 0.00488
    ...                         ...
    recruitment_flux           (time, fish, X) float64 1.453e-09 ... 0.0
    fish_catch_rate            (time, fish, X) float64 0.0 0.0 ... 0.0008219
    encounter_rate_link        (time, feeding_link, X) float64 2.288 ... 0.0
    encounter_rate_total       (time, fish, X) float64 2.288 2.288 ... 7.132e-07
    consumption_rate_max_pred  (time, fish, X) float64 0.2375 0.2375 ... 0.01062
    consumption_rate_link      (time, feeding_link, X) float64 0.2152 ... 0.0

In [7]:
if nsteps > 1:
    check_key = f"day{nsteps}"
    test_ds = testcase.ds.isel(X=0, time=-1)
    if make_plot:
        for group in test_ds.group.data:
            testcase.ds.biomass.sel(group=group).isel(X=0).plot()
        plt.ylim([1e-6, 1])
        plt.yscale("log")
        plt.legend(test_ds.group.data)
else:
    check_key = "day1"
    test_ds = testcase.ds.isel(X=0, time=0)

In [8]:
compare(
    baselines[check_key]["T_habitat"],
    test_ds["T_habitat"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Sp | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Sd | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Mf | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Mp | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Md | 4.2000e+00 | 4.2000e+00 | 0.0000e+00 |
| Lp | 1.8199e+01 | 1.8199e+01 | 0.0000e+00 |
| Ld | 4.2414e+00 | 4.2414e+00 | 0.0000e+00 |


In [9]:
compare(
    baselines[check_key]["metabolism"],
    test_ds["metabolism_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 4.2960e-02 | 4.2960e-02 | 0.0000e+00 |
| Sp | 4.2960e-02 | 4.2960e-02 | 0.0000e+00 |
| Sd | 4.2960e-02 | 4.2960e-02 | 0.0000e+00 |
| Mf | 1.4479e-02 | 1.4479e-02 | 2.3961e-16 |
| Mp | 1.4479e-02 | 1.4479e-02 | 2.3961e-16 |
| Md | 4.3744e-03 | 4.3744e-03 | 0.0000e+00 |
| Lp | 4.8801e-03 | 4.8801e-03 | 0.0000e+00 |
| Ld | 1.4796e-03 | 1.4796e-03 | 2.9311e-16 |


In [10]:
compare(
    baselines[check_key]["encounter"],
    test_ds["encounter_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.2936e+00 | 2.2936e+00 | 1.9362e-16 |
| Sp_Zoo | 2.2936e+00 | 2.2936e+00 | 1.9362e-16 |
| Sd_Zoo | 2.2936e+00 | 2.2936e+00 | 1.9362e-16 |
| Mf_Zoo | 2.9781e-01 | 2.9781e-01 | 5.5920e-16 |
| Mf_Sf | 1.9835e-06 | 1.9835e-06 | 2.1352e-16 |
| Mf_Sp | 1.9832e-06 | 1.9832e-06 | 6.4067e-16 |
| Mf_Sd | 1.9832e-06 | 1.9832e-06 | 6.4067e-16 |
| Mp_Zoo | 2.9781e-01 | 2.9781e-01 | 5.5920e-16 |
| Mp_Sf | 1.9835e-06 | 1.9835e-06 | 2.1352e-16 |
| Mp_Sp | 1.9832e-06 | 1.9832e-06 | 6.4067e-16 |
| Mp_Sd | 1.9832e-06 | 1.9832e-06 | 6.4067e-16 |
| Md_benthic_prey | 3.4382e-04 | 3.4382e-04 | 3.1534e-16 |
| Lp_Mf | 3.1696e-07 | 3.1696e-07 | 1.6702e-16 |
| Lp_Mp | 6.3392e-07 | 6.3392e-07 | 0.0000e+00 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 9.8667e-08 | 9.8667e-08 | 0.0000e+00 |
| Ld_Mp | 1.9733e-07 | 1.9733e-07 | 1.3414e-16 |
| Ld_Md | 2.6202e-07 | 2.6202e-07 | 0.0000e+00 |
| Ld_benthic_prey | 

In [11]:
compare(
    baselines[check_key]["consumption"],
    test_ds["consumption_rate_link"],
    markdown_formatting=True,
)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.1523e-01 | 2.1523e-01 | 1.2896e-16 |
| Sp_Zoo | 2.1523e-01 | 2.1523e-01 | 1.2896e-16 |
| Sd_Zoo | 2.1523e-01 | 2.1523e-01 | 1.2896e-16 |
| Mf_Zoo | 4.2980e-02 | 4.2980e-02 | 1.6145e-16 |
| Mf_Sf | 2.8625e-07 | 2.8625e-07 | 1.8494e-16 |
| Mf_Sp | 2.8621e-07 | 2.8621e-07 | 1.8497e-16 |
| Mf_Sd | 2.8621e-07 | 2.8621e-07 | 1.8497e-16 |
| Mp_Zoo | 4.2980e-02 | 4.2980e-02 | 1.6145e-16 |
| Mp_Sf | 2.8625e-07 | 2.8625e-07 | 1.8494e-16 |
| Mp_Sp | 2.8621e-07 | 2.8621e-07 | 1.8497e-16 |
| Mp_Sd | 2.8621e-07 | 2.8621e-07 | 1.8497e-16 |
| Md_benthic_prey | 3.3822e-04 | 3.3822e-04 | 1.6028e-16 |
| Lp_Mf | 3.1693e-07 | 3.1693e-07 | 0.0000e+00 |
| Lp_Mp | 6.3387e-07 | 6.3387e-07 | 0.0000e+00 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 9.6478e-08 | 9.6478e-08 | 0.0000e+00 |
| Ld_Mp | 1.9296e-07 | 1.9296e-07 | 1.3718e-16 |
| Ld_Md | 2.5621e-07 | 2.5621e-07 | 2.0663e-16 |
| Ld_benthic_prey | 

In [12]:
compare(
    baselines[check_key]["ingestion"],
    test_ds["ingestion_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.1523e-01 | 2.1523e-01 | 1.2896e-16 |
| Sp | 2.1523e-01 | 2.1523e-01 | 1.2896e-16 |
| Sd | 2.1523e-01 | 2.1523e-01 | 1.2896e-16 |
| Mf | 4.2981e-02 | 4.2981e-02 | 1.6144e-16 |
| Mp | 4.2981e-02 | 4.2981e-02 | 1.6144e-16 |
| Md | 3.3822e-04 | 3.3822e-04 | 1.6028e-16 |
| Lp | 9.5080e-07 | 9.5080e-07 | 0.0000e+00 |
| Ld | 9.7803e-05 | 9.7803e-05 | 0.0000e+00 |


In [13]:
compare(
    baselines[check_key]["predation"],
    test_ds["predation_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 6.3415e-07 | 6.3415e-07 | 1.6696e-16 |
| Sp | 6.3415e-07 | 6.3415e-07 | 0.0000e+00 |
| Sd | 6.3415e-07 | 6.3415e-07 | 0.0000e+00 |
| Mf | 3.7569e-07 | 3.7569e-07 | 1.4091e-16 |
| Mp | 7.5139e-07 | 7.5139e-07 | 0.0000e+00 |
| Md | 2.3169e-07 | 2.3169e-07 | 1.1425e-16 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [14]:
compare(
    baselines[check_key]["mortality"],
    test_ds["mortality_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Sd | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mf | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Mp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Md | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Lp | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |
| Ld | 2.7397e-04 | 2.7397e-04 | 1.9787e-16 |


In [15]:
compare(
    baselines[check_key]["fishing"],
    test_ds["fish_catch_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 8.2192e-04 | 8.2192e-04 | 1.3191e-16 |
| Mp | 8.2192e-05 | 8.2192e-05 | 0.0000e+00 |
| Md | 8.2192e-05 | 8.2192e-05 | 0.0000e+00 |
| Lp | 8.2192e-04 | 8.2192e-04 | 0.0000e+00 |
| Ld | 8.2192e-04 | 8.2192e-04 | 0.0000e+00 |


In [16]:
compare(
    baselines[check_key]["avail_energy"],
    test_ds["energy_avail_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0770e-01 | 1.0770e-01 | 3.8656e-16 |
| Sp | 1.0770e-01 | 1.0770e-01 | 3.8656e-16 |
| Sd | 1.0770e-01 | 1.0770e-01 | 3.8656e-16 |
| Mf | 1.5607e-02 | 1.5607e-02 | 5.5575e-16 |
| Mp | 1.5607e-02 | 1.5607e-02 | 5.5575e-16 |
| Md | -4.1377e-03 | -4.1377e-03 | 2.0963e-16 |
| Lp | -4.8794e-03 | -4.8794e-03 | -0.0000e+00 |
| Ld | -1.4111e-03 | -1.4111e-03 | 3.0733e-16 |


In [17]:
compare(
    baselines[check_key]["growth"],
    test_ds["growth_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.0765e-01 | 1.0765e-01 | 2.5784e-16 |
| Sp | 1.0765e-01 | 1.0765e-01 | 2.5784e-16 |
| Sd | 1.0765e-01 | 1.0765e-01 | 2.5784e-16 |
| Mf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mp | 1.5286e-02 | 1.5286e-02 | 3.4046e-16 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [18]:
compare(
    baselines[check_key]["reproduction"],
    test_ds["reproduction_rate"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.4543e-02 | 1.4543e-02 | 2.3856e-16 |
| Mp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Lp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [19]:
compare(
    baselines[check_key]["recruitment"],
    test_ds["recruitment_flux"],
    markdown_formatting=True,
)

| fish | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 1.6108e-09 | 1.6108e-09 | 5.1353e-16 |
| Sp | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Sd | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Mf | 1.0764e-06 | 1.0764e-06 | 1.9673e-16 |
| Mp | 1.0762e-06 | 1.0762e-06 | 1.9676e-16 |
| Md | 1.0762e-06 | 1.0762e-06 | 1.9676e-16 |
| Lp | 1.6930e-07 | 1.6930e-07 | 4.6905e-16 |
| Ld | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |


In [20]:
compare(
    baselines[check_key]["biomass"],
    test_ds["biomass"],
    markdown_formatting=True,
)

| group | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf | 9.9987e-06 | 9.9987e-06 | 0.0000e+00 |
| Sp | 9.9956e-06 | 9.9956e-06 | 0.0000e+00 |
| Sd | 9.9956e-06 | 9.9956e-06 | 0.0000e+00 |
| Mf | 1.2152e-05 | 1.2152e-05 | 2.7882e-16 |
| Mp | 1.2152e-05 | 1.2152e-05 | 0.0000e+00 |
| Md | 1.2057e-05 | 1.2057e-05 | 4.2152e-16 |
| Lp | 1.0202e-05 | 1.0202e-05 | 3.3210e-16 |
| Ld | 9.9495e-06 | 9.9495e-06 | 0.0000e+00 |
| benthic_prey | 4.1870e-03 | 4.1870e-03 | 0.0000e+00 |
